## Assignment 04 - Human vs AI Chess Game
## Minimax Algorithm

### Tehreem Zafar
### 22i-1630
### CY-A

In [4]:
import pygame
import sys
from abc import ABC, abstractmethod
from collections import namedtuple

# Initialize Pygame
pygame.init()

# Constants
BOARD_SIZE = 8
SQUARE_SIZE = 80
WIDTH, HEIGHT = BOARD_SIZE * SQUARE_SIZE, BOARD_SIZE * SQUARE_SIZE
FPS = 60

# Colors
WHITE = (245, 245, 220)
GRAY = (119, 148, 85)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Setup display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Human vs AI Chess")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)
small_font = pygame.font.SysFont(None, 28)

# Move representation
Move = namedtuple('Move', 'sr sc er ec piece captured')

# Piece Classes
class Piece(ABC):
    def __init__(self, color):
        self.color = color
        self.has_moved = False
        
    @property
    def opponent_color(self):
        return 'black' if self.color == 'white' else 'white'
    
    @abstractmethod
    def get_symbol(self):
        pass
    
    @abstractmethod
    def get_moves(self, board, r, c):
        pass
    
    def __str__(self):
        return f"{self.get_symbol()}{self.color.capitalize()}"
    
    def __repr__(self):
        return self.__str__()

class Pawn(Piece):
    def get_symbol(self):
        return 'P'
    
    def get_moves(self, board, r, c):
        moves = []
        d = -1 if self.color == 'white' else 1
        
        # Move forward one square
        if board.empty(r+d, c):
            moves.append(Move(r, c, r+d, c, self, None))
            
            # Initial two-square move
            if (r == 6 and self.color == 'white') or (r == 1 and self.color == 'black'):
                if board.empty(r+2*d, c):
                    moves.append(Move(r, c, r+2*d, c, self, None))
            
        # Capture diagonally
        for dc in (-1, 1):
            if board.in_bounds(r+d, c+dc) and board.get_piece(r+d, c+dc) and board.get_piece(r+d, c+dc).color != self.color:
                moves.append(Move(r, c, r+d, c+dc, self, board.get_piece(r+d, c+dc)))
                
        return moves

class Knight(Piece):
    def get_symbol(self):
        return 'N'
    
    def get_moves(self, board, r, c):
        moves = []
        for dr, dc in [(-2,-1), (-2,1), (-1,-2), (-1,2), (1,-2), (1,2), (2,-1), (2,1)]:
            if board.in_bounds(r+dr, c+dc):
                target = board.get_piece(r+dr, c+dc)
                if not target or target.color != self.color:
                    moves.append(Move(r, c, r+dr, c+dc, self, target))
        return moves

class Bishop(Piece):
    def get_symbol(self):
        return 'B'
    
    def get_moves(self, board, r, c):
        return self._get_sliding_moves(board, r, c, [(1,1), (-1,-1), (1,-1), (-1,1)])
    
    def _get_sliding_moves(self, board, r, c, directions):
        moves = []
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            while board.in_bounds(nr, nc):
                target = board.get_piece(nr, nc)
                if not target:
                    moves.append(Move(r, c, nr, nc, self, None))
                elif target.color != self.color:
                    moves.append(Move(r, c, nr, nc, self, target))
                    break
                else:
                    break
                nr += dr
                nc += dc
        return moves

class Rook(Piece):
    def get_symbol(self):
        return 'R'
    
    def get_moves(self, board, r, c):
        return self._get_sliding_moves(board, r, c, [(1,0), (-1,0), (0,1), (0,-1)])
    
    def _get_sliding_moves(self, board, r, c, directions):
        moves = []
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            while board.in_bounds(nr, nc):
                target = board.get_piece(nr, nc)
                if not target:
                    moves.append(Move(r, c, nr, nc, self, None))
                elif target.color != self.color:
                    moves.append(Move(r, c, nr, nc, self, target))
                    break
                else:
                    break
                nr += dr
                nc += dc
        return moves

class Queen(Piece):
    def get_symbol(self):
        return 'Q'
    
    def get_moves(self, board, r, c):
        directions = [(1,0), (-1,0), (0,1), (0,-1), (1,1), (-1,-1), (1,-1), (-1,1)]
        moves = []
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            while board.in_bounds(nr, nc):
                target = board.get_piece(nr, nc)
                if not target:
                    moves.append(Move(r, c, nr, nc, self, None))
                elif target.color != self.color:
                    moves.append(Move(r, c, nr, nc, self, target))
                    break
                else:
                    break
                nr += dr
                nc += dc
        return moves

class King(Piece):
    def get_symbol(self):
        return 'K'
    
    def get_moves(self, board, r, c):
        moves = []
        for dr in (-1, 0, 1):
            for dc in (-1, 0, 1):
                # Skip the current position
                if dr == 0 and dc == 0:
                    continue
                    
                if board.in_bounds(r+dr, c+dc):
                    target = board.get_piece(r+dr, c+dc)
                    if not target or target.color != self.color:
                        moves.append(Move(r, c, r+dr, c+dc, self, target))
        return moves

# Board Class
class Board:
    def __init__(self):
        self.grid = [[None]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        self.init_board()
        
    def init_board(self):
        # Black back row
        self.grid[0][0] = Rook('black')
        self.grid[0][1] = Knight('black')
        self.grid[0][2] = Bishop('black')
        self.grid[0][3] = Queen('black')
        self.grid[0][4] = King('black')
        self.grid[0][5] = Bishop('black')
        self.grid[0][6] = Knight('black')
        self.grid[0][7] = Rook('black')
        
        # Black pawns
        for c in range(8):
            self.grid[1][c] = Pawn('black')
            
        # White pawns
        for c in range(8):
            self.grid[6][c] = Pawn('white')
            
        # White back row
        self.grid[7][0] = Rook('white')
        self.grid[7][1] = Knight('white')
        self.grid[7][2] = Bishop('white')
        self.grid[7][3] = Queen('white')
        self.grid[7][4] = King('white')
        self.grid[7][5] = Bishop('white')
        self.grid[7][6] = Knight('white')
        self.grid[7][7] = Rook('white')
    
    def in_bounds(self, r, c):
        return 0 <= r < 8 and 0 <= c < 8
    
    def empty(self, r, c):
        return self.in_bounds(r, c) and not self.grid[r][c]
    
    def get_piece(self, r, c):
        if not self.in_bounds(r, c):
            return None
        return self.grid[r][c]
    
    def set_piece(self, r, c, piece):
        if self.in_bounds(r, c):
            self.grid[r][c] = piece
    
    def find_king(self, color):
        for r in range(8):
            for c in range(8):
                piece = self.get_piece(r, c)
                if piece and isinstance(piece, King) and piece.color == color:
                    return r, c
        return None
    
    def get_all_pieces(self, color):
        pieces = []
        for r in range(8):
            for c in range(8):
                piece = self.get_piece(r, c)
                if piece and piece.color == color:
                    pieces.append((r, c, piece))
        return pieces

# Player Classes
class Player(ABC):
    def __init__(self, color):
        self.color = color
    
    @abstractmethod
    def get_move(self, game):
        pass

class HumanPlayer(Player):
    def __init__(self, color):
        super().__init__(color)
        self.selected = None
        self.valid_moves = []
    
    def get_move(self, game):
        # Handled by event processing in game loop
        return None
    
    def handle_click(self, game, pos):
        c, r = pos[0] // SQUARE_SIZE, pos[1] // SQUARE_SIZE
        
        # If a piece is already selected
        if self.selected:
            for move in self.valid_moves:
                if (move.er, move.ec) == (r, c):
                    # Valid move selected, execute it
                    self.selected = None
                    self.valid_moves = []
                    return move
            
            # Click wasn't on a valid move - either deselect or select a new piece
            self.selected = None
            self.valid_moves = []
            
            # Check if we're selecting a new piece
            piece = game.board.get_piece(r, c)
            if piece and piece.color == self.color:
                self.selected = (r, c)
                self.valid_moves = game.get_legal_moves(self.color, r, c)
        else:
            # No piece selected yet - try to select one
            piece = game.board.get_piece(r, c)
            if piece and piece.color == self.color:
                self.selected = (r, c)
                self.valid_moves = game.get_legal_moves(self.color, r, c)
        
        return None

class AIPlayer(Player):
    def __init__(self, color, depth=3):
        super().__init__(color)
        self.depth = depth
        self.thinking = False
        self.move = None
    
    def get_move(self, game):
        if not self.thinking and not self.move:
            # Start thinking in the next frame
            self.thinking = True
            return None
        
        if self.thinking:
            # Calculate the best move
            _, move = self.minimax(game, self.depth, float('-inf'), float('inf'), self.color == 'white')
            self.move = move
            self.thinking = False
            return None
        
        if self.move:
            # Return the calculated move
            move = self.move
            self.move = None
            return move
        
        return None
    
    def minimax(self, game, depth, alpha, beta, maximizing):
        if depth == 0:
            return self.evaluate(game), None
        
        if game.checkmate(game.current_player):
            return float('-inf') if maximizing else float('inf'), None
        
        if game.stalemate(game.current_player):
            return 0, None
        
        best_move = None
        if maximizing:
            max_eval = float('-inf')
            for move in game.get_all_legal_moves('white'):
                game.make_move(move)
                eval_score, _ = self.minimax(game, depth - 1, alpha, beta, False)
                game.undo_move()
                
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = move
                
                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break
            
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in game.get_all_legal_moves('black'):
                game.make_move(move)
                eval_score, _ = self.minimax(game, depth - 1, alpha, beta, True)
                game.undo_move()
                
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
                
                beta = min(beta, eval_score)
                if beta <= alpha:
                    break
            
            return min_eval, best_move
    
    def evaluate(self, game):
        values = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 1000}
        score = 0
        
        for r in range(8):
            for c in range(8):
                piece = game.board.get_piece(r, c)
                if piece:
                    value = values[piece.get_symbol()]
                    score += value if piece.color == 'white' else -value
        
        return score

# Game Manager Class
class ChessGame:
    def __init__(self):
        self.board = Board()
        self.current_player = 'white'
        self.white_player = HumanPlayer('white')
        self.black_player = AIPlayer('black', depth=2)
        self.over = False
        self.result = ''
        self.history = []  # stack of (move, captured, player)
        self.load_images()
    
    def load_images(self):
        self.piece_images = {}
        pieces = ["P", "R", "N", "B", "Q", "K"]
        for piece in pieces:
            for color in ["white", "black"]:
                try:
                    img = pygame.image.load(f"chess_pieces/{piece}{color.capitalize()}.png")
                    self.piece_images[f"{piece}{color.capitalize()}"] = pygame.transform.scale(img, (SQUARE_SIZE, SQUARE_SIZE))
                except pygame.error:
                    print(f"Could not load image: chess_pieces/{piece}{color.capitalize()}.png")
                    # Create a placeholder
                    img_surface = pygame.Surface((SQUARE_SIZE, SQUARE_SIZE), pygame.SRCALPHA)
                    pygame.draw.rect(img_surface, (255, 0, 0, 128), (0, 0, SQUARE_SIZE, SQUARE_SIZE))
                    text = small_font.render(f"{piece}", True, (0, 0, 0))
                    img_surface.blit(text, (SQUARE_SIZE//2 - text.get_width()//2, SQUARE_SIZE//2 - text.get_height()//2))
                    self.piece_images[f"{piece}{color.capitalize()}"] = img_surface
    
    def make_move(self, move):
        if not move:
            return False
            
        # Save history
        captured = self.board.get_piece(move.er, move.ec)
        self.history.append((move, captured, self.current_player))
        
        # Move piece
        self.board.set_piece(move.er, move.ec, move.piece)
        self.board.set_piece(move.sr, move.sc, None)
        
        # Pawn promotion
        if isinstance(move.piece, Pawn) and (move.er == 0 or move.er == 7):
            self.board.set_piece(move.er, move.ec, Queen(move.piece.color))
        
        # Switch turn
        self.current_player = 'black' if self.current_player == 'white' else 'white'
        
        # Check for end game
        self.check_end_game()
        
        return True
    
    def undo_move(self):
        if not self.history:
            return
        
        move, captured, prev_turn = self.history.pop()
        self.board.set_piece(move.sr, move.sc, move.piece)
        self.board.set_piece(move.er, move.ec, captured)
        self.current_player = prev_turn
        self.over = False
        self.result = ''

    # ---------- add the two helpers here ----------
    def _push_move(self, move):
        """Execute move without history or end‑of‑game logic (used only for testing)."""
        captured = self.board.get_piece(move.er, move.ec)
        self.board.set_piece(move.er, move.ec, move.piece)
        self.board.set_piece(move.sr, move.sc, None)
        return captured

    def _pop_move(self, move, captured):
        """Undo the move done by _push_move()."""
        self.board.set_piece(move.sr, move.sc, move.piece)
        self.board.set_piece(move.er, move.ec, captured)
    # ---------------------------------------------
    
    def get_all_moves(self, color):
        moves = []
        for r, c, piece in self.board.get_all_pieces(color):
            moves.extend(piece.get_moves(self.board, r, c))
        return moves
    
    def get_all_legal_moves(self, color):
        all_moves = []
        for r, c, piece in self.board.get_all_pieces(color):
            all_moves.extend(self.get_legal_moves(color, r, c))
        return all_moves
    
    def get_legal_moves(self, color, r, c):
        piece = self.board.get_piece(r, c)
        if not piece or piece.color != color:
            return []

        legal_moves = []
        potential_moves = piece.get_moves(self.board, r, c)

        for move in potential_moves:
            captured_piece = self._push_move(move)      # ← use helper
            if not self.in_check(color):
                legal_moves.append(move)
            self._pop_move(move, captured_piece)        # ← use helper

        return legal_moves
    
    def in_check(self, color):
        king_pos = self.board.find_king(color)
        if not king_pos:
            return False
        
        kr, kc = king_pos
        opponent_color = 'black' if color == 'white' else 'white'
        
        for move in self.get_all_moves(opponent_color):
            if (move.er, move.ec) == (kr, kc):
                return True
        
        return False
    
    def checkmate(self, color):
        return not self.get_all_legal_moves(color) and self.in_check(color)
    
    def stalemate(self, color):
        return not self.get_all_legal_moves(color) and not self.in_check(color)
    
    def check_end_game(self):
        if self.checkmate(self.current_player):
            winner = 'White' if self.current_player == 'black' else 'Black'
            self.result = f'Checkmate! {winner} wins'
            self.over = True
        elif self.stalemate(self.current_player):
            self.result = 'Stalemate!'
            self.over = True
    
    def get_current_player(self):
        return self.white_player if self.current_player == 'white' else self.black_player
    
    def get_evaluation(self):
        values = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 1000}
        score = 0
        
        for r in range(8):
            for c in range(8):
                piece = self.board.get_piece(r, c)
                if piece:
                    value = values[piece.get_symbol()]
                    score += value if piece.color == 'white' else -value
        
        return score
    
    def draw(self):
        # Draw board
        for r in range(8):
            for c in range(8):
                color = WHITE if (r + c) % 2 == 0 else GRAY
                pygame.draw.rect(screen, color, (c * SQUARE_SIZE, r * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
        
        # Draw highlights for selected piece and valid moves
        human_player = self.white_player if isinstance(self.white_player, HumanPlayer) and self.current_player == 'white' else None
        if human_player and human_player.selected:
            r, c = human_player.selected
            pygame.draw.rect(screen, BLUE, (c * SQUARE_SIZE, r * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 4)
            
            for move in human_player.valid_moves:
                pygame.draw.circle(screen, GREEN, 
                                 (move.ec * SQUARE_SIZE + SQUARE_SIZE // 2, 
                                  move.er * SQUARE_SIZE + SQUARE_SIZE // 2), 8)
        
        # Draw pieces
        for r in range(8):
            for c in range(8):
                piece = self.board.get_piece(r, c)
                if piece:
                    img_key = f"{piece.get_symbol()}{piece.color.capitalize()}"
                    if img_key in self.piece_images:
                        screen.blit(self.piece_images[img_key], (c * SQUARE_SIZE, r * SQUARE_SIZE))
        
        # Check indicator
        if not self.over and self.in_check(self.current_player):
            text = small_font.render('Check!', True, RED)
            screen.blit(text, (10, 10))
        
        # Score/eval
        score = self.get_evaluation()
        txt = small_font.render(f'Score: {score}', True, RED)
        screen.blit(txt, (10, 40))
        
        # Current player
        player_txt = small_font.render(f'Turn: {self.current_player.capitalize()}', True, RED)
        screen.blit(player_txt, (10, 70))
        
        # End game message
        if self.over:
            # Draw semi-transparent overlay
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 128))
            screen.blit(overlay, (0, 0))
            
            # Draw message
            msg = font.render(self.result, True, RED)
            rect = msg.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(msg, rect)
            
            # Draw restart message
            restart_msg = small_font.render("Press R to restart", True, WHITE)
            restart_rect = restart_msg.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 50))
            screen.blit(restart_msg, restart_rect)

def main():
    game = ChessGame()
    
    while True:
        clock.tick(FPS)
        
        # Process events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            
            # Handle key presses
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r and game.over:
                    # Restart game
                    game = ChessGame()
                elif event.key == pygame.K_u:
                    # Undo move
                    game.undo_move()
            
            # Handle human player input
            if not game.over and game.current_player == 'white' and event.type == pygame.MOUSEBUTTONDOWN:
                move = game.white_player.handle_click(game, event.pos)
                if move:
                    game.make_move(move)
        
        # Handle AI move
        if not game.over and game.current_player == 'black':
            move = game.black_player.get_move(game)
            if move:
                game.make_move(move)
        
        # Draw everything
        game.draw()
        pygame.display.flip()

if __name__ == '__main__':
    main()

SystemExit: 